In [55]:
import csv
import numpy as np
import pandas as pd
from pathlib import Path

В этом блокноте будут мержиться несколько UCI-датасетов в один объединённый.

In [56]:
uci_collections = [(Path("../datasets/postnauka"), "pn"),
                   (Path("../datasets/ruwiki"), "ruwiki")]

g_path, g_collection_name = Path("../datasets"), "merged"

Объединим словари в один и сохраним его.

In [63]:
%%time

global_dictionary = {}
local_mappings = []

for path, collection_name in uci_collections:
    local_mapping = []
    with (path / ("vocab.%s.txt" % collection_name)).open() as infile:
        for ix, line in enumerate(infile):
            token, modality = line.split()
            value = global_dictionary.get((token, modality))
            if value is None:
                value = global_dictionary[(token, modality)] = len(global_dictionary)
            local_mapping.append(value)
    local_mappings.append(local_mapping)

global_mapping = sorted(map(lambda p: (p[1], p[0]), global_dictionary.items()))

# Добавим метки коллекций в словарь
collection_ids = {}
for _, collection_name in uci_collections:
    collection_ids[collection_name] = len(global_mapping)
    global_mapping.append((len(global_mapping), (collection_name, "collection_id")))

with (g_path / ("vocab.%s.txt" % g_collection_name)).open("w") as outfile:
    for _, value in global_mapping:
        outfile.write("%s %s\n" % value)

CPU times: user 2.81 s, sys: 47 ms, total: 2.86 s
Wall time: 2.86 s


Теперь перестроим документы по объединённому словарю.

Сначала посчитаем суммарную длину и кол-во документов будущего `docword` файла.

In [66]:
g_doc_count, g_bow_length = 0, 0

for path, collection_name in uci_collections:
    with (path / ("docword.%s.txt" % collection_name)).open() as infile:
        dict_length = infile.readline()
        doc_count = int(infile.readline())
        bow_length = int(infile.readline())
        g_doc_count += doc_count
        g_bow_length += bow_length

Теперь построим `docword` файл объединённой коллекции.

In [69]:
%%time

with (g_path / ("docword.%s.txt" % g_collection_name)).open("w") as outfile:
    g_doc_id = 0
    outfile.write("%d\n%d\n%d\n" % (len(global_dictionary), g_doc_count, g_bow_length))
    for mapping, (path, collection_name) in zip(local_mappings, uci_collections):
        with (path / ("docword.%s.txt" % collection_name)).open() as infile:
            dict_length, doc_count, bow_length = int(infile.readline()), int(infile.readline()), int(infile.readline())
            seen_docs = set()
            for line in infile:
                doc_id, word_id, word_count = map(int, line.split())
                if doc_id not in seen_docs:
                    # Добавим метки коллекций в документы
                    collection_id = collection_ids[collection_name]
                    outfile.write("%d %d %d\n" % (g_doc_id + doc_id, collection_id, 1))
                    seen_docs.add(doc_id)
                g_word_id = mapping[word_id - 1] + 1
                outfile.write("%d %d %d\n" % (g_doc_id + doc_id, g_word_id, word_count))
            g_doc_id += doc_count

CPU times: user 13min 36s, sys: 5.78 s, total: 13min 42s
Wall time: 13min 44s


---